<a href="https://colab.research.google.com/github/Slrosales/Huesitos/blob/main/C%C3%B3digo/Huesitos_Interactivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install scikit-image
!pip install datasets
!pip install --upgrade gradio
!pip install tensorflow
clear_output()

In [ ]:
import numpy as np
import cv2
from PIL import Image
import gradio as gr
from scipy.ndimage import gaussian_filter
from skimage import img_as_ubyte
import tensorflow as tf
from datasets import load_dataset

# Extraccion de Dataset desde Huuging Face

In [ ]:
dataset = load_dataset('LosHuesitos9-9/Huesitos')
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/626 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/44 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/88 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 626
    })
    validation: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 44
    })
    test: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 88
    })
})

In [ ]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [ ]:
dataset['train'][100]

{'image_id': 271,
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x640>,
 'width': 640,
 'height': 640,
 'objects': {'id': [350],
  'area': [336],
  'bbox': [[485.0, 326.0, 21.0, 16.0]],
  'category': [2]}}

In [ ]:
dataset['train'][400]

{'image_id': 575,
 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=420x512>,
 'width': 420,
 'height': 512,
 'objects': {'id': [], 'area': [], 'bbox': [], 'category': []}}

Información sobre la estructura del dataset:

- **ID de la imagen**: Cada entrada tiene un identificador único asociado con la clave `image_id`.
- **URL de la imagen**: Las imágenes están alojadas en línea y se puede acceder a ellas mediante el enlace proporcionado en `image['src']`.
- **Dimensiones de la imagen**: Cada imagen tiene dimensiones especificadas por `width` y `height`.
- **Objetos en la imagen**: Hay objetos en las imágenes (que probablemente sean referencias a fracturas u otras características) que tienen:
  - Su propio **ID**, indicado por la clave `id`.
  - **Área** del objeto, indicada por la clave `area`.
  - **Cuadro delimitador** (`bbox`) que especifica la ubicación y el tamaño del objeto en la imagen.
  - **Categoría** del objeto, indicada por la clave `category`.


In [ ]:
train_data = dataset['train']

# Modelo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load
**Solo ejecutar si ya se tiene el archivo del modelo**

In [ ]:
# Cargar el modelo guardado
# modificar el path de ser necesario
modelo = tf.keras.models.load_model("/content/drive/MyDrive/YOLOv4_training/+87.h5")


# Gradio

## Preprocesamiento

In [ ]:
# Imagenes del dataset
image0 = dataset['train'][200]['image'].save("0.jpg")
image1 = dataset['train'][218]['image'].save("1.jpg")
image3 = dataset['train'][234]['image'].save("2.jpg")
image4 = dataset['train'][213]['image'].save("3.jpg")
image5 = dataset['train'][217]['image'].save("4.jpg")
image6 = dataset['train'][439]['image'].save("5.jpg")
image7 = dataset['train'][487]['image'].save("6.jpg")
image8 = dataset['train'][403]['image'].save("7.jpg")
image8 = dataset['train'][404]['image'].save("8.jpg")
# Otras imagenes

In [ ]:
def preprocess_image3(input_image, target_size=(224, 224)):
    # Comprobar si input_image es una imagen PIL y convertirla a un array NumPy
    if isinstance(input_image, Image.Image):
        image_np = np.array(input_image)
    elif isinstance(input_image, np.ndarray):  # Ya es un array NumPy
        image_np = input_image
    else:
        raise TypeError("El input no es ni una imagen PIL ni un array NumPy.")

    # Convertir a escala de grises si es una imagen a color
    if image_np.ndim == 3:
        image_gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    elif image_np.ndim == 2:
        image_gray = image_np
    else:
        raise ValueError("El array de la imagen tiene un número de canales no soportado.")

    # Aplicar CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image_clahe = clahe.apply(image_gray)

    # Redimensionar la imagen en escala de grises
    image_resized = cv2.resize(image_clahe, target_size, interpolation=cv2.INTER_AREA)

    # Normalizar el array de la imagen a [0, 1]
    normalized_image = image_resized / 255.0

    return normalized_image

## Gradiente

### Otsu

In [ ]:
def metodo_otsu(magnitude):
  histogram, bins = np.histogram(magnitude.flatten(), bins=256, range=[0,256])
  bins = bins[0:-1] # bin_edges tiene un elemento más que el array hist

  probabilidades = histogram / np.sum(histogram)
  media_global = np.sum(bins * probabilidades)

  sigma_b_cuadrada_max = 0
  umbral_optimo = 0

  for t in range(len(bins)):
      P1 = np.sum(probabilidades[:t])
      P2 = np.sum(probabilidades[t:])
      if P1 == 0 or P2 == 0:
          continue
      M1 = np.sum(bins[:t] * probabilidades[:t]) / P1
      M2 = np.sum(bins[t:] * probabilidades[t:]) / P2
      sigma_b_cuadrada = P1 * (M1 - media_global)**2 + P2 * (M2 - media_global)**2
      if sigma_b_cuadrada > sigma_b_cuadrada_max:
          sigma_b_cuadrada_max = sigma_b_cuadrada
          umbral_optimo = bins[t]

  return umbral_optimo

### Gradiente

In [ ]:
def sigma_adaptativo(imagen, factor=0.15):
  varianza = np.var(imagen)

  # Calcular sigma como una función logarítmica de la varianza para evitar que sea demasiado grande
  sigma = np.log1p(varianza) * factor

  return sigma

In [ ]:
def central_difference(image):
  # Convertir la imagen a float64 para evitar desbordamiento durante la resta
  image = image.astype(np.float64)

  height, width = image.shape

  Gx = np.zeros((height, width), dtype=np.float64)
  Gy = np.zeros((height, width), dtype=np.float64)

  # Iterar sobre la imagen evitando los bordes
  for i in range(1, height-1):
      for j in range(1, width-1):

        # Diferencias centrales
        Gx[i, j] = (image[i, j+1] - image[i, j-1]) / 2
        Gy[i, j] = (image[i+1, j] - image[i-1, j]) / 2


  return Gx, Gy

In [ ]:
def gradiente3(image):
  gray_image = img_as_ubyte(image)

  Gx, Gy = central_difference(gray_image)

  angle = np.arctan2(Gy, Gx)

  magnitud = np.sqrt(Gx**2 + Gy**2)

  T =  metodo_otsu(magnitud)

  binary_image = (magnitud > T).astype(np.uint8) * 255

  # Normalizar la magnitud para la visualización
  normalized_magnitud = (magnitud / magnitud.max()) * 255
  normalized_magnitud = normalized_magnitud.astype(np.uint8)

  # Angulo visual
  angle_degrees = np.degrees(angle)
  angle_normalized = (angle_degrees + 180) / 360  # Normalizar los ángulos entre 0 y 1
  angle_for_display = (angle_normalized * 255).astype(np.uint8)  # Escalar para el rango de 8 bits

  # Aplicar filtro gaussiano para suavizado
  sigma = sigma_adaptativo(image)
  radiografia_suavizada = gaussian_filter(image, sigma=sigma)

  binarizado_gradiente = magnitud > T

  # Restar la imagen binarizada de bordes de la radiografía suavizada
  bordes_resaltados = radiografia_suavizada - binarizado_gradiente

  return Gx, Gy, normalized_magnitud, binary_image, bordes_resaltados

## Predicción

In [ ]:
def predict_fracture3(model, image):
    # Preprocesamiento de la imagen
    image_processed = preprocess_image3(image)
    image_processed_2d = np.squeeze(image_processed)  # Guardar versión de 2 dimensiones

    # Expandir las dimensiones para la predicción del modelo
    image_for_model = np.expand_dims(image_processed, axis=0)
    image_for_model = np.expand_dims(image_for_model, axis=-1)

    prediction = modelo.predict(image_for_model)

    no_identificado = 0
    # Probabilidad de fractura
    probability_of_fracture = prediction[0][0]
    if (abs((1 - probability_of_fracture)-probability_of_fracture) <= 0.25):
      no_identificado = 1

    response = {
        'No Fractura':  1 - probability_of_fracture,
        'Fractura': probability_of_fracture,
        'No identificado': no_identificado
    }


    Gx, Gy, magnitud, binary_image, bordes_resaltados = gradiente3(image_processed_2d)

    def to_uint8(img):
        img_normalized = (img - img.min()) / (img.max() - img.min())
        # Escalar y convertir a uint8
        return (255 * img_normalized).astype(np.uint8)


    # Convertir a formato de imagen para mostrar en Gradio
    output_images = [
        Image.fromarray(to_uint8(image_processed_2d), mode="L"),
        Image.fromarray(to_uint8(Gx), mode="L"),
        Image.fromarray(to_uint8(Gy), mode="L"),
        Image.fromarray(to_uint8(magnitud), mode="L"),
        Image.fromarray(to_uint8(binary_image), mode="L"),
        Image.fromarray(to_uint8(bordes_resaltados), mode="L")
    ]

    return [response] + output_images

# Interactivo

In [ ]:
# Gradio interfaz
interface = gr.Interface(
    fn=lambda image: predict_fracture3(modelo, image),
    inputs=gr.Image(type="pil"),
    outputs=[gr.Label(num_top_classes=3)] + [gr.Image(type="pil") for _ in range(6)],
    examples=["0.jpg", "1.jpg", "2.jpg", "3.jpg", "4.jpg", "5.jpg", "6.jpg", "7.jpg", "8.jpg"],
    live=False
)

interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://54d6374985b414e4e4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
